[View in Colaboratory](https://colab.research.google.com/github/tcglarry/geo/blob/master/geo_Unet_colab_correct_add_ch64_gram.ipynb)

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

aoi_aoi_test1.ipynb  Sample upload.txt			      Untitled0.ipynb
aoi_test	     TAHU PowerPoint Template.pdf	      Untitled1.ipynb
Colab Notebooks      TAHU PowerPoint Template - 柱狀圖 1.ods
PiXNet		     test for excel


In [5]:

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Conv2DTranspose
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import regularizers

Using TensorFlow backend.


In [0]:
src = 'drive/aoi_test/t_brain/geo/'

In [7]:
data= np.load(src+'train_data.npy')
label = np.load(src+'train_labels.npy')
print (np.mean(label))
label = label[:,:,:,np.newaxis]
print (data.shape)
print (label.shape)

5.66184075511905
(4200, 21, 21, 20)
(4200, 20, 20, 1)


In [0]:

data_mean = np.mean(data)
data_std = np.std(data)
label_max = np.max(label)
label_min = np.min(label)

train_data = (data - data_mean)/data_std
#label_data = (label-label_min)/(label_max - label_min)
label_data = np.log(label)

In [0]:
train_x,val_x,train_y,val_y = train_test_split(train_data,label_data,test_size=0.2, shuffle=True)

In [0]:
def conv_block(ch, activation= 'relu', padding='same',kernel_regularizer=regularizers.l2(0.01)):       
    return Conv2D(ch,(3,3),activation= activation, padding =padding )


In [0]:
#def build_model_1(IMAGE_HEIGHT=IMAGE_HEIGHT,IMAGE_WIDTH=IMAGE_WIDTH,ch=ch):
def build_model_1(ch=64):
    #inputs = Input((IMAGE_HEIGHT,IMAGE_WIDTH,ch))
    inputs = Input((21,21,20))

    conv0 = Conv2D(32,(2,2),padding='valid')(inputs)
    print ('conv0',conv0.get_shape())
    conv1 =  conv_block(ch)(conv0)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    print ('pool1',pool1.get_shape())
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    print ('poo12',pool2.get_shape())
    
    conv_test = Conv2D(64,(2,2),padding='valid')(pool2)
    print ('conv_test',conv_test.get_shape())
    
    
    
    conv3 =  conv_block(ch*4)(conv_test)
    conv3 = conv_block(ch*4)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    print ('poo13',pool3.get_shape())
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    print ('poo14',pool4.get_shape())
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)

    
    up6 = concatenate([UpSampling2D(size=(2,2))(conv5), conv4], axis=3)
    conv6 = conv_block(ch*8)(up6)
    conv6 = conv_block(ch*8)(conv6)
    
    up7 = concatenate ([UpSampling2D(size=(2,2))(conv6), conv3],  axis=3)
    conv7 = conv_block(ch*4)(up7)
    conv7 = conv_block(ch*4)(conv7)
    print ('conv7',conv7.get_shape())
    conv7 = Conv2DTranspose(ch*4,(2,2),padding='valid')(conv7)
    print ('conv7',conv7.get_shape())
    
    up8 = concatenate([UpSampling2D(size=(2,2))(conv7), conv2], axis=3)
    conv8 = conv_block(ch*2)(up8)
    conv8 = conv_block(ch*2)(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2,2))(conv8), conv1],  axis=3)
    conv9 = conv_block(ch)(up9)
    conv9 = conv_block(ch)(conv9)
    
    
    
    conv10 = Conv2D(1,(1,1))(conv9)
    
  
    model = Model(inputs= inputs, outputs=conv10)
    
    return model
    
    

In [62]:
model = build_model_1()
model.summary()

conv0 (?, 20, 20, 32)
pool1 (?, 10, 10, 64)
poo12 (?, 5, 5, 128)
conv_test (?, 4, 4, 64)
poo13 (?, 2, 2, 256)
poo14 (?, 1, 1, 512)
conv7 (?, 4, 4, 256)
conv7 (?, ?, ?, 256)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 21, 21, 20)   0                                            
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 20, 20, 32)   2592        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 20, 20, 64)   18496       conv2d_22[0][0]                  
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (No

In [0]:



def gram_matrix(x):
  #features = K.batch_flatten(K.permute_dimensions(x[0], (2, 0, 1)))
  x = K.batch_flatten(x)
  gram = K.dot(x, K.transpose(x))
  print(gram)
  #return K.expand_dims(gram, axis=0)
  return gram
  


def my_loss(y_true,y_pred):
  loss_1 = K.mean(K.square(y_pred - y_true), axis=-1)
  
  #loss_2= K.categorical_crossentropy(gram_matrx(y_true), gram_matrix(y_pred))
  loss_2 = K.mean(K.square(gram_matrix(y_pred) - gram_matrix(y_true)))
  
  total_loss = loss_1 * 1.0 + loss_2 **0.0001
  return total_loss
  



In [64]:
#model.compile (optimizer='Adam', loss = 'mse', metrics =['mae'])
model.compile (optimizer='Adam', loss = my_loss, metrics =['mae'])

Tensor("loss_9/conv2d_42_loss/MatMul:0", shape=(?, ?), dtype=float32)
Tensor("loss_9/conv2d_42_loss/MatMul_1:0", shape=(?, ?), dtype=float32)


In [0]:

# train the model on the new data for a few epochs

use_model= False
if use_model:
    

    if os.path.isfile(src+'water_first_try_2.h5'):

      model = load_model(src+'water_first_try_2.h5')
      print ('model laoded')
    else:
      print ('model not exist')


In [66]:

checkpoint = ModelCheckpoint(filepath=src+'water_first_try_2_prepro_ch64_gram.h5', monitor = 'val_loss', save_best_only=True, mode= 'auto')
earlystop = EarlyStopping(patience=10)
callback_list = [checkpoint, earlystop]
model.fit(x=train_x, y=train_y, batch_size=32, epochs=300, callbacks = callback_list, verbose=1, validation_split=0.2,  shuffle=True)


Train on 2688 samples, validate on 672 samples
Epoch 1/300
2688/2688 [==============================] - 18s 7ms/step - loss: 2.4784 - mean_absolute_error: 0.8098 - val_loss: 1.4922 - val_mean_absolute_error: 0.5515
Epoch 2/300
2688/2688 [==============================] - 13s 5ms/step - loss: 1.3701 - mean_absolute_error: 0.4748 - val_loss: 1.2784 - val_mean_absolute_error: 0.4101
Epoch 3/300
2688/2688 [==============================] - 13s 5ms/step - loss: 1.2427 - mean_absolute_error: 0.3825 - val_loss: 1.2077 - val_mean_absolute_error: 0.3526
Epoch 4/300
2688/2688 [==============================] - 13s 5ms/step - loss: 1.1944 - mean_absolute_error: 0.3413 - val_loss: 1.1735 - val_mean_absolute_error: 0.3224
Epoch 5/300
2688/2688 [==============================] - 13s 5ms/step - loss: 1.1611 - mean_absolute_error: 0.3103 - val_loss: 1.1482 - val_mean_absolute_error: 0.2966
Epoch 6/300
2688/2688 [==============================] - 13s 5ms/step - loss: 1.1354 - mean_absolute_error: 0.283

In [67]:
error = model.evaluate(val_x,val_y)
print (error)

840/840 [==============================] - 1s 1ms/step
[1.0249645006088983, 0.11103879866145906]


In [68]:

print ('label mean = ', np.mean(label_data))
print ('per_element_error = ', np.sqrt(error[0]/840.))
print ('error percentage = ', str(round(np.sqrt(error[0]/400.)/np.mean(label_data)*100.,2))+'%')
     

label mean =  1.2508584707231873
per_element_error =  0.03493130163704485
error percentage =  4.05%


In [0]:
!cp drive/aoi_test/t_brain/geo/test2.npy .

In [70]:
print (os.path.isfile('test2.npy'))
test = np.load('test2.npy')
test_data = (test - data_mean)/data_std
test_data.shape

True


(1800, 21, 21, 20)

In [71]:
prediction = model.predict(test_data)
prediction = np.exp(prediction)
pred = prediction.reshape(1800,-1)
print (pred.shape)


pred_df = pd.DataFrame(pred)
pred_df.keys()


(1800, 400)


RangeIndex(start=0, stop=400, step=1)

In [0]:
test_df = pd.read_csv(src+'upload_sample.csv')
col_names = test_df.keys().tolist()

In [73]:
print (src)

drive/aoi_test/t_brain/geo/


In [74]:
final = pd.concat([test_df['ID'],pred_df],axis=1)
final.columns = col_names
final.to_csv(src+'test_submit_0831_ch64_gram.csv', index=False)
print ('submit file saved')

submit file saved


In [75]:
test_see_df = pd.read_csv(src+'test_submit_0831_ch32_gram.csv')
test_see_df

,ID,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,...,loc391,loc392,loc393,loc394,loc395,loc396,loc397,loc398,loc399,loc400
0,F0000009,2.075976,2.339811,0.686893,0.533440,0.507373,0.497232,0.550936,0.799921,1.116335,...,19.612362,8.473093,2.989978,5.507019,5.437169,2.834534,3.667053,4.262979,6.203764,3.954820
1,F0000011,5.740894,5.234048,3.320109,2.051649,1.532063,2.508738,1.749931,1.457652,0.927044,...,1.925525,1.775311,1.386481,1.577252,1.367470,0.571080,0.826093,0.412720,0.369276,0.535941
2,F0000012,5.054601,5.769695,3.788895,1.657590,0.502485,0.426970,0.672344,0.431218,0.546094,...,1.577249,2.541003,1.990372,2.078974,1.484417,0.632783,0.793938,1.289706,1.356385,0.871878
3,F0000014,1.735199,2.167517,1.133670,0.555450,0.248450,0.688942,2.085719,1.819201,0.962582,...,2.608430,3.030443,8.309262,10.615378,11.830481,7.101485,6.313229,4.454718,2.785871,4.050359
4,F0000017,0.202533,0.432206,0.657352,0.923454,0.596499,0.471972,1.680032,2.697525,2.402415,...,7.204242,19.775710,28.050215,19.505861,13.326053,3.821878,1.721020,0.796852,2.990088,1.246225
5,F0000018,4.187602,5.644590,2.957388,1.919224,1.058897,0.462887,0.786464,0.855868,1.560453,...,2.483816,3.278805,0.899624,1.653245,3.814007,6.102831,3.004096,1.571642,3.611474,5.911630
6,F0000024,1.041673,0.964459,1.582050,1.438068,1.081712,0.806367,0.702651,1.017895,1.695813,...,1.134551,2.080022,3.707790,2.809410,3.077346,3.800334,2.234962,2.425806,3.608616,2.689387
7,F0000036,5.332394,5.731726,3.549970,8.248824,5.867391,6.020679,7.029463,5.013000,6.625993,...,6.567584,3.333193,4.082226,6.981921,11.826679,17.025282,21.961960,7.645431,6.131884,5.505541
8,F0000046,7.161613,11.832506,25.720295,13.952134,5.841220,2.801017,1.712101,2.152633,3.442361,...,5.874712,7.541568,7.283820,3.532268,4.038185,8.053838,7.721926,6.717470,5.203308,6.962462
9,F0000047,1.779934,1.991217,2.895393,3.575640,2.572719,1.553703,1.798219,1.606855,1.969549,...,6.691837,8.955525,6.192682,4.037485,6.058806,7.775933,6.991693,9.738720,13.975223,11.179706


In [0]:
!cp drive/aoi_test/t_brain/geo/test_submit.csv .

In [0]:
from google.colab import files
files.download('test_submit.csv')